Implementing Grid Search on keras, based on this <a href="https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/tutorial"> tutorial</a> 

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_iris
from sklearn.model_selection import GridSearchCV
from tensorflow import keras
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

In [2]:
iris = load_iris()
X = iris["data"]
y = iris["target"]

The class variable is very imbalanced.

We split the dataset into train and test. We shuffle the examples as the order is not important here.

In [3]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y = le.fit_transform(y)

n_class_var = len(np.unique(y))

y = keras.utils.to_categorical(y, num_classes=n_class_var)

We scale with the Standard Scaler. That is the features are normalized. Models usually work better when this is the case.

In [4]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

The model's architecture. 

In [5]:
def create_model(optimizer="sgd"):
    model = keras.Sequential(
        [
            keras.layers.Dense(
                8, activation="relu", input_shape=(X.shape[-1],)
            ),
            keras.layers.Dense(12, activation="relu"),
            keras.layers.Dense(8, activation="relu"),
            keras.layers.Dense(n_class_var, activation="softmax")
        ]
    )
    model.compile(
        optimizer="adam",
        loss="categorical_crossentropy",
        metrics=['categorical_accuracy']
    )
    return model

In [6]:
model = KerasClassifier(build_fn=create_model, verbose=0)
# define the grid search parameters
batch_size = [1, 4, 8]
epochs = [10, 20]
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)
grid_result = grid.fit(X, y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

C:\Users\flis1\Miniconda3\envs\tensorflow\lib\site-packages\sklearn\model_selection\_validation.py:774: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\flis1\Miniconda3\envs\tensorflow\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\flis1\Miniconda3\envs\tensorflow\lib\site-packages\sklearn\metrics\_scorer.py", line 418, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
  File "C:\Users\flis1\Miniconda3\envs\tensorflow\lib\site-packages\tensorflow\python\keras\wrappers\scikit_learn.py", line 308, in score
    raise ValueError('The model is not configured to compute accuracy. '
ValueError: The model is not configured to compute accuracy. You should pass `metrics=["accuracy"]` to the `model.compile()` method.

  UserWarning,
C:\Users\flis1\Miniconda3\envs\t

C:\Users\flis1\Miniconda3\envs\tensorflow\lib\site-packages\sklearn\model_selection\_validation.py:774: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\flis1\Miniconda3\envs\tensorflow\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\flis1\Miniconda3\envs\tensorflow\lib\site-packages\sklearn\metrics\_scorer.py", line 418, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
  File "C:\Users\flis1\Miniconda3\envs\tensorflow\lib\site-packages\tensorflow\python\keras\wrappers\scikit_learn.py", line 308, in score
    raise ValueError('The model is not configured to compute accuracy. '
ValueError: The model is not configured to compute accuracy. You should pass `metrics=["accuracy"]` to the `model.compile()` method.

  UserWarning,
C:\Users\flis1\Miniconda3\envs\t

Best: nan using {'batch_size': 1, 'epochs': 10}
nan (nan) with: {'batch_size': 1, 'epochs': 10}
nan (nan) with: {'batch_size': 1, 'epochs': 20}
nan (nan) with: {'batch_size': 4, 'epochs': 10}
nan (nan) with: {'batch_size': 4, 'epochs': 20}
nan (nan) with: {'batch_size': 8, 'epochs': 10}
nan (nan) with: {'batch_size': 8, 'epochs': 20}
